In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.appName('reg').getOrCreate()

In [ ]:
import io
%cd '/Users/rajeshprabhakar/Desktop/Hackathon/BlackFridaySales'

/Users/rajeshprabhakar/Desktop/Hackathon/BlackFridaySales


In [ ]:
salestrain=pd.read_csv('train.csv')

In [ ]:
salestest=pd.read_csv('test.csv')

In [ ]:
salestraindf=spark.createDataFrame(salestrain)

In [ ]:
salestestdf=spark.createDataFrame(salestest)

In [ ]:
from pyspark.sql.functions import isnan,isnull,when,count,col

In [ ]:
salestraindf.select([count(when(isnan(c),c)).alias(c)
                    for c in salestraindf.columns]).show()

+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender|Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|      0|         0|     0|  0|         0|            0|                         0|             0|                 0|            173638|            383247|       0|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+



In [ ]:
salestestdf.select([count(when(isnan(c),c)).alias(c)
                   for c in salestestdf.columns]).show()

+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+
|User_ID|Product_ID|Gender|Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+
|      0|         0|     0|  0|         0|            0|                         0|             0|                 0|             72344|            162562|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+



In [ ]:
salestraindf=salestraindf.na.fill(value=999,
                                 subset=['Product_Category_2'])

In [ ]:
salestraindf=salestraindf.na.fill(value=888,
                                 subset=['Product_category_3'])

In [ ]:
salestestdf=salestestdf.na.fill(value=999,
                               subset=['Product_Category_2'])

In [ ]:
salestestdf=salestestdf.na.fill(value=888,
                               subset=['Product_Category_3'])

In [ ]:
salestraindf.printSchema()

root
 |-- User_ID: long (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: long (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: long (nullable = true)
 |-- Product_Category_1: long (nullable = true)
 |-- Product_Category_2: double (nullable = false)
 |-- Product_Category_3: double (nullable = false)
 |-- Purchase: long (nullable = true)



In [ ]:
salestraindf.select(['User_ID','Product_ID']).show(5)

+-------+----------+
|User_ID|Product_ID|
+-------+----------+
|1000001| P00069042|
|1000001| P00248942|
|1000001| P00087842|
|1000001| P00085442|
|1000002| P00285442|
+-------+----------+
only showing top 5 rows



In [ ]:
salestraindf.select(['City_Category',
                     'Stay_In_Current_City_Years']).show(5)

+-------------+--------------------------+
|City_Category|Stay_In_Current_City_Years|
+-------------+--------------------------+
|            A|                         2|
|            A|                         2|
|            A|                         2|
|            A|                         2|
|            C|                        4+|
+-------------+--------------------------+
only showing top 5 rows



In [ ]:
salestraindf.groupBy('Stay_In_Current_City_Years').count().show()

+--------------------------+------+
|Stay_In_Current_City_Years| count|
+--------------------------+------+
|                         3| 95285|
|                         0| 74398|
|                        4+| 84726|
|                         1|193821|
|                         2|101838|
+--------------------------+------+



In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

In [ ]:
indexers=[StringIndexer(inputCol=column,
                       outputCol=column+'_index').fit(salestraindf)
         for column in list(set(salestraindf.columns)-
                           set(['Purchase','User_ID','Product_ID']))]

In [ ]:
pipeline=Pipeline(stages=indexers)
salestraindf=pipeline.fit(salestraindf).transform(salestraindf)

In [ ]:
salestestdf=pipeline.fit(salestestdf).transform(salestestdf)

In [ ]:
salestraindf.columns

['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3',
 'Purchase',
 'Gender_index',
 'Stay_In_Current_City_Years_index',
 'Product_Category_1_index',
 'Age_index',
 'City_Category_index',
 'Product_Category_2_index',
 'Marital_Status_index',
 'Product_Category_3_index',
 'Occupation_index']

In [ ]:
columnstodrop=['User_ID','Product_ID','Gender','Age','Occupation',
 'City_Category','Stay_In_Current_City_Years','Marital_Status',
 'Product_Category_1','Product_Category_2','Product_Category_3',]

In [ ]:
salestraindf=salestraindf.drop(*columnstodrop)

In [ ]:
from pyspark.ml.feature import RFormula

In [ ]:
formula=RFormula(formula='Purchase~.',featuresCol='features',
                labelCol='label')

In [ ]:
output=formula.fit(salestraindf).transform(salestraindf)
output.select('features','label').show(5)

+--------------------+-------+
|            features|  label|
+--------------------+-------+
|[1.0,1.0,6.0,6.0,...| 8370.0|
|[1.0,1.0,1.0,6.0,...|15200.0|
|[1.0,1.0,12.0,6.0...| 1422.0|
|[1.0,1.0,12.0,6.0...| 1057.0|
|[0.0,3.0,2.0,5.0,...| 7969.0|
+--------------------+-------+
only showing top 5 rows



In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
linreg=LinearRegression(featuresCol='features',labelCol='label')

In [ ]:
linregmodel=linreg.fit(output)

In [ ]:
linregmodel.summary.r2adj

0.06356300117930158

In [ ]:
linregmodel.summary.rootMeanSquaredError

4860.75986981341

In [ ]:
salestestdf=salestestdf.drop(*columnstodrop)

In [ ]:
salestestdf.columns

['Gender_index',
 'Stay_In_Current_City_Years_index',
 'Product_Category_1_index',
 'Age_index',
 'City_Category_index',
 'Product_Category_2_index',
 'Marital_Status_index',
 'Product_Category_3_index',
 'Occupation_index']

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler=VectorAssembler(inputCols=['Age_index',
'Product_Category_2_index','Product_Category_1_index',
 'Stay_In_Current_City_Years_index','Occupation_index',
 'Gender_index','City_Category_index',
 'Product_Category_3_index','Marital_Status_index'],
                         outputCol='features')

In [ ]:
testoutput=assembler.transform(salestestdf)

In [ ]:
testoutput.select('features').show(5)

+--------------------+
|            features|
+--------------------+
|[3.0,9.0,1.0,1.0,...|
|[0.0,6.0,6.0,4.0,...|
|[1.0,2.0,0.0,3.0,...|
|[1.0,12.0,7.0,3.0...|
|[0.0,6.0,7.0,0.0,...|
+--------------------+
only showing top 5 rows



In [ ]:
regpredict=linregmodel.transform(testoutput)

In [ ]:
regpredict.select('prediction').show(5)

+-----------------+
|       prediction|
+-----------------+
| 6391.54807827454|
| 8593.91385040738|
|7723.127430105531|
|8091.752748068837|
|11907.51851717382|
+-----------------+
only showing top 5 rows



In [ ]:
regpredict.select('prediction').toPandas().to_csv('reg.csv')

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor

In [ ]:
tree=DecisionTreeRegressor(featuresCol='features',labelCol='label')

In [ ]:
treemodel=tree.fit(output)

In [ ]:
treepredict=treemodel.transform(testoutput)

In [ ]:
treepredict.select('prediction').toPandas().to_csv('tree.csv')

In [ ]:
lr = LinearRegression(regParam=0.1,elasticNetParam=0.9)

In [ ]:
lrmodel=lr.fit(output)

In [ ]:
lrmodel.summary.rootMeanSquaredError

4860.759876976344

In [ ]:
from pyspark.ml.regression import GBTRegressor

In [ ]:
gbm=GBTRegressor()

In [ ]:
gbmmodel=gbm.fit(output)

In [ ]:
gbmpredict=gbmmodel.transform(testoutput)

In [ ]:
gbmpredict.select('prediction').toPandas().to_csv('gbm.csv')